In [1]:
#https://www.kaggle.com/datasets/gyejr95/league-of-legendslol-ranked-games-2020-ver1


<h1> ETL Processes </h1>

Use this notebook to develop the ETL process for each of your tables before completing the etl.py file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *
import json # data processing, json file I/O
from sqlalchemy import Table, Column, Float, Integer, String, MetaData, ForeignKey, JSON,Boolean
import sqlalchemy # copy pd dataframe to Redshift 
from sqlalchemy.types import * # load staging_tables
from sqlalchemy import create_engine

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=MOBAtatic user=postgres password=123456")
cur = conn.cursor()

In [3]:
#take a look through our datasets file
for dirname, _, filenames in os.walk('archive/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

archive/match_data_version1.pickle
archive/challenger_match_V2.csv
archive/match_loser_data_version1.csv
archive/match_loser_data_version1.pickle
archive/challenger_match.csv
archive/match_winner_data_version1.csv
archive/match_data_version1.csv
archive/match_data_version2.pickle
archive/match_winner_data_version1.pickle


<b>Describe and Gather Data </b> <br>
The Kaggle dataset contains 10,800 game data in the KR server, the Data Dragon (meta_champs & meta_items) contains all the in-game data up to patch 10.15.1, which is the latest version of game by Aug. 2, 2020.

Read 3 match data pickle files, match_df contains participants for both the winning and losing side of a match; winnder_df contains in-game stats and objetives info for the winning team of a match; loser_df contains the same info for the losing side

In [4]:
match_df = pd.read_pickle('archive/match_data_version1.pickle')
winner_df =  pd.read_pickle('archive/match_loser_data_version1.pickle')
loser_df = pd.read_pickle('archive/match_winner_data_version1.pickle')

In [5]:
print(match_df.info())
match_df.iloc[0]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108829 entries, 0 to 0
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   gameCreation           108829 non-null  float64
 1   gameDuration           108829 non-null  float64
 2   gameId                 108829 non-null  float64
 3   gameMode               108829 non-null  object 
 4   gameType               108829 non-null  object 
 5   gameVersion            108829 non-null  object 
 6   mapId                  108829 non-null  float64
 7   participantIdentities  108829 non-null  object 
 8   participants           108829 non-null  object 
 9   platformId             108829 non-null  object 
 10  queueId                108829 non-null  float64
 11  seasonId               108829 non-null  float64
 12  status.message         0 non-null       object 
 13  status.status_code     0 non-null       float64
dtypes: float64(7), object(7)
memory usage: 12

gameCreation                                               1585154663829.0
gameDuration                                                        1323.0
gameId                                                        4247263043.0
gameMode                                                           CLASSIC
gameType                                                      MATCHED_GAME
gameVersion                                                  10.6.314.4405
mapId                                                                 11.0
participantIdentities    [{'participantId': 1, 'player': {'platformId':...
participants             [{'participantId': 1, 'teamId': 100, 'champion...
platformId                                                              KR
queueId                                                              420.0
seasonId                                                              13.0
status.message                                                         NaN
status.status_code       

In [6]:
print(winner_df.info())
winner_df.iloc[0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108829 entries, 0 to 108828
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   teamId                108829 non-null  int64  
 1   win                   108817 non-null  object 
 2   firstBlood            108829 non-null  bool   
 3   firstTower            108829 non-null  bool   
 4   firstInhibitor        108829 non-null  bool   
 5   firstBaron            108829 non-null  bool   
 6   firstDragon           108829 non-null  bool   
 7   firstRiftHerald       108829 non-null  bool   
 8   towerKills            108829 non-null  int64  
 9   inhibitorKills        108829 non-null  int64  
 10  baronKills            108829 non-null  int64  
 11  dragonKills           108829 non-null  int64  
 12  vilemawKills          108829 non-null  int64  
 13  riftHeraldKills       108829 non-null  int64  
 14  dominionVictoryScore  108829 non-null  int64  
 15  

teamId                                                                100
win                                                                  Fail
firstBlood                                                           True
firstTower                                                          False
firstInhibitor                                                      False
firstBaron                                                          False
firstDragon                                                         False
firstRiftHerald                                                     False
towerKills                                                              0
inhibitorKills                                                          0
baronKills                                                              0
dragonKills                                                             0
vilemawKills                                                            0
riftHeraldKills                       

In [7]:
print(loser_df.info())
loser_df.iloc[0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108829 entries, 0 to 108828
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   teamId                108829 non-null  int64  
 1   win                   108829 non-null  object 
 2   firstBlood            108829 non-null  bool   
 3   firstTower            108829 non-null  bool   
 4   firstInhibitor        108829 non-null  bool   
 5   firstBaron            108829 non-null  bool   
 6   firstDragon           108829 non-null  bool   
 7   firstRiftHerald       108829 non-null  bool   
 8   towerKills            108829 non-null  int64  
 9   inhibitorKills        108829 non-null  int64  
 10  baronKills            108829 non-null  int64  
 11  dragonKills           108829 non-null  int64  
 12  vilemawKills          108829 non-null  int64  
 13  riftHeraldKills       108829 non-null  int64  
 14  dominionVictoryScore  108829 non-null  int64  
 15  

teamId                                                                200
win                                                                   Win
firstBlood                                                          False
firstTower                                                           True
firstInhibitor                                                       True
firstBaron                                                          False
firstDragon                                                          True
firstRiftHerald                                                      True
towerKills                                                              9
inhibitorKills                                                          1
baronKills                                                              0
dragonKills                                                             3
vilemawKills                                                            0
riftHeraldKills                       

In [123]:
meta_champs = pd.read_json('en_US-10.15.1/meta_champion.json')


In [124]:
meta_champs

,type,format,version,data
Aatrox,champion,standAloneComplex,10.15.1,"{'version': '10.15.1', 'id': 'Aatrox', 'key': ..."
Ahri,champion,standAloneComplex,10.15.1,"{'version': '10.15.1', 'id': 'Ahri', 'key': '1..."
Akali,champion,standAloneComplex,10.15.1,"{'version': '10.15.1', 'id': 'Akali', 'key': '..."
Alistar,champion,standAloneComplex,10.15.1,"{'version': '10.15.1', 'id': 'Alistar', 'key':..."
Amumu,champion,standAloneComplex,10.15.1,"{'version': '10.15.1', 'id': 'Amumu', 'key': '..."
...,...,...,...,...
Zed,champion,standAloneComplex,10.15.1,"{'version': '10.15.1', 'id': 'Zed', 'key': '23..."
Ziggs,champion,standAloneComplex,10.15.1,"{'version': '10.15.1', 'id': 'Ziggs', 'key': '..."
Zilean,champion,standAloneComplex,10.15.1,"{'version': '10.15.1', 'id': 'Zilean', 'key': ..."
Zoe,champion,standAloneComplex,10.15.1,"{'version': '10.15.1', 'id': 'Zoe', 'key': '14..."


the only info we need from the meta_champs is, the key-name mapping of each champion, becuase it is used in all match tables above.

similarly, we also need the 'key-name' mapping of each item from meta_items

In [125]:
with open('en_US-10.15.1/meta_item.json') as f:
    data = json.load(f)
meta_items = pd.read_json(json.dumps(data['data']), orient='index')

In [126]:
meta_items.head()

,name,description,colloq,plaintext,into,image,gold,tags,maps,stats,...,depth,effect,hideFromAll,stacks,consumed,inStore,consumeOnFull,specialRecipe,requiredChampion,requiredAlly
1001,Boots of Speed,<groupLimit>Limited to 1 pair of boots.</group...,;,Slightly increases Movement Speed,"[3006, 3047, 3020, 3158, 3111, 3117, 3009, 4001]","{'full': '1001.png', 'sprite': 'item0.png', 'g...","{'base': 300, 'purchasable': True, 'total': 30...",[Boots],"{'11': True, '12': True, '21': True, '22': False}",{'FlatMovementSpeedMod': 25},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1004,Faerie Charm,<stats><mana>+25% Base Mana Regen </mana></stats>,;,Slightly increases Mana Regen,"[3028, 3070, 3073, 3114]","{'full': '1004.png', 'sprite': 'item0.png', 'g...","{'base': 125, 'purchasable': True, 'total': 12...",[ManaRegen],"{'11': True, '12': True, '21': True, '22': False}",{},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1006,Rejuvenation Bead,<stats>+50% Base Health Regen </stats>,;,Slightly increases Health Regen,"[3077, 3801, 3194, 4401]","{'full': '1006.png', 'sprite': 'item0.png', 'g...","{'base': 150, 'purchasable': True, 'total': 15...",[HealthRegen],"{'11': True, '12': True, '21': True, '22': False}",{},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1011,Giant's Belt,<stats>+380 Health</stats>,;,Greatly increases Health,"[3083, 3084, 3022, 3143, 3742]","{'full': '1011.png', 'sprite': 'item0.png', 'g...","{'base': 600, 'purchasable': True, 'total': 10...",[Health],"{'11': True, '12': True, '21': True, '22': False}",{'FlatHPPoolMod': 380},...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1018,Cloak of Agility,<stats>+20% Critical Strike Chance</stats>,;,Increases critical strike chance,"[3086, 3095, 3031, 3508]","{'full': '1018.png', 'sprite': 'item0.png', 'g...","{'base': 800, 'purchasable': True, 'total': 80...",[CriticalStrike],"{'11': True, '12': True, '21': True, '22': False}",{'FlatCritChanceMod': 0.2},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
meta_items.info()
meta_items.iloc[0]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264 entries, 1001 to 4403
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              264 non-null    object 
 1   description       264 non-null    object 
 2   colloq            264 non-null    object 
 3   plaintext         264 non-null    object 
 4   into              84 non-null     object 
 5   image             264 non-null    object 
 6   gold              264 non-null    object 
 7   tags              264 non-null    object 
 8   maps              264 non-null    object 
 9   stats             264 non-null    object 
 10  from              165 non-null    object 
 11  depth             165 non-null    float64
 12  effect            212 non-null    object 
 13  hideFromAll       26 non-null     float64
 14  stacks            16 non-null     float64
 15  consumed          19 non-null     float64
 16  inStore           34 non-null     float6

name                                                   Boots of Speed
description         <groupLimit>Limited to 1 pair of boots.</group...
colloq                                                              ;
plaintext                           Slightly increases Movement Speed
into                 [3006, 3047, 3020, 3158, 3111, 3117, 3009, 4001]
image               {'full': '1001.png', 'sprite': 'item0.png', 'g...
gold                {'base': 300, 'purchasable': True, 'total': 30...
tags                                                          [Boots]
maps                {'11': True, '12': True, '21': True, '22': False}
stats                                    {'FlatMovementSpeedMod': 25}
from                                                              NaN
depth                                                             NaN
effect                                                            NaN
hideFromAll                                                       NaN
stacks              

In [13]:
match_df['gameMode'].unique()

array(['CLASSIC', 'ARAM', 'ONEFORALL', 'URF', 'TUTORIAL_MODULE_3',
       'TUTORIAL_MODULE_1', 'KINGPORO', 'TUTORIAL_MODULE_2'], dtype=object)

Only classic games are needed, we should drop all other game modes

In [14]:
gameId_CLASSIC =  match_df[match_df['gameMode'] == 'CLASSIC'].gameId.tolist()

In [15]:
winner_df = winner_df[winner_df['gameId'].isin(gameId_CLASSIC)]
loser_df = loser_df[loser_df['gameId'].isin(gameId_CLASSIC)]
match_df = match_df[match_df['gameId'].isin(gameId_CLASSIC)]

In [16]:
match_df.gameMode.unique()

array(['CLASSIC'], dtype=object)

if one player disconnected from a game right from the start, the team can choose to <b>remake</b> the game, meaning the match will end immediately <br>

remake should not be regarded the same as a loss, because the other players in the team just did not want a 4v5 situation<br>

we should identify the remake games from match_df, as they affect our win/loss analysis<br>

a team can only choose to reamke in the first 15 mins of the game, after that the reamke attempt becomes an early surrender(which is just the same as a loss)<br>

also in reality, rarely a game at this ranking level would finish before 15 mins, unless someone disconnected
<br>
so if gameDuration in match_df <= 15 mins, we can almost be certain that the game is a remake<br>

we will drop these games<br>

In [17]:
gameId_remake = match_df.query('gameDuration <= 15*60').gameId.values.tolist()

In [18]:
winner_df = winner_df[~winner_df['gameId'].isin(gameId_remake)]
loser_df = loser_df[~loser_df['gameId'].isin(gameId_remake)]
match_df = match_df[~match_df['gameId'].isin(gameId_remake)]


In [19]:
print('now the shortest game in df is {:.2f} mins'.format(match_df.gameDuration.min() / 60))

now the shortest game in df is 15.02 mins


In [20]:
# in the meta_dfs, we only need the key-name mapping, all other columns are dropped
meta_items = meta_items[['name']]
meta_champs['key'] = meta_champs.apply(lambda row: row.data['key'], axis=1)
meta_champs = meta_champs[['key']]

In [21]:
for query in drop_table_queries:
        cur.execute(query)
#create tables
for query in create_table_queries:
        cur.execute(query)

In [22]:
cur.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cur.fetchall():
    print(table)

('games',)
('champions',)
('items',)
('objectives_visions',)
('champion_key',)
('item_key',)


In [23]:
pd.options.display.max_rows = 75
cur.execute("""SELECT table_name, column_name, data_type FROM information_schema.columns WHERE table_name LIKE 'staging_%'""")
pd.DataFrame(cur.fetchall(), columns=['table_name', 'column_name', 'data_type'])

,table_name,column_name,data_type


<h1> #1: champions Table </h1>

champ1 - champion 10 is stored in staging_match, we can retreive them and the gameId of each 10 champX

the champions_table_value statement looks a bit complicated, due to the nature of json array column in psotgres

1. a.explode("participants") basically duplicates each row in the participants N times, where N is the number of items (here this is 10) in the array (if any) of participants of that row. <br>

2. Since explode duplicates the rows, the original rows' indices (0 and 1) are copied to the new rows, so their indices are 0, 0, 1, 1, which messes up later processing. reset_index() creates a fresh new column for the index, starting at 0. drop=True is used because by default pandas will keep the old index column; this removes it. <br>

3. pd.json_normalize(df['participants']) converts the column (where each row contains a JSON object) to a new dataframe where each key unique of all the JSON objects is new column. <br>

4. df.merge() merges the new dataframe into the original one <br>
5. left_index=True and right_index=True tells pandas to merge the specified dataframe starting from it's first, row into this dataframe, starting at its first row <br>
6. we just need three column ['gameId','participantId','championId'] <br>
7. make a pivot here to convert the participantID to column name which is consist which the champions table (champ_1, champ_2, champ_3,...champ_10) <br>
8. reset_index to avoid the sub index after applying pivot and then remove the index column name by using rename_axis <br>

In [87]:
a = match_df[['gameId', 'participants']]

In [88]:
a = match_df[['gameId', 'participants']]
df = a.explode('participants').reset_index(drop=True)
df = df.merge(pd.json_normalize(df['participants']), left_index=True, right_index=True)

In [26]:
champions_df = df[['gameId','participantId','championId'  ]]
champions_df['idx'] = champions_df.groupby('gameId').cumcount()
b = champions_df.pivot(index = 'gameId', columns='idx', values = 'championId').reset_index().rename_axis(columns=None, axis=1)

/var/folders/rs/xjf_6_vd4j16cdjg_j2td15h0000gn/T/ipykernel_27914/1481404577.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  champions_df['idx'] = champions_df.groupby('gameId').cumcount()


We see the index is 20, let check for the unique of gameID and also the reason behind why there are 20 index in this case instead of 10 as our expectation.

In [27]:
is_20 = b[b[11].notnull()].gameId
print('Number of records contain 20 champ_id: ', len(is_20))
duplicate = a[a.duplicated('gameId')]
is_dup = b[b.duplicated('gameId')].gameId
print('Number of records duplicate champ_id: ', len(is_dup))

Number of records contain 20 champ_id:  34
Number of records duplicate champ_id:  0


In [89]:
#drop duplicate in gameid
b = b.drop_duplicates(subset=['gameId'])
#drop column with contain chapm number is 20
b = b[~b['gameId'].isin(is_20)]
#also exclude from orginal df
df = df[~df['gameId'].isin(is_20)]
#just select column from 0->9
c = b.iloc[:, 0:11]

one issue is noted before the insertion can be completed: some games in the original data have mssing champ_ids

In [90]:
#just exclude the game that contain the number of champ < 10
is_miss = c[c.isnull().any(axis=1)].gameId
c = c[~c['gameId'].isin(is_miss)]
#also exclude from orginal df
df = df[~df['gameId'].isin(is_miss)]

In [30]:
#convert type to valid int
c = c.astype(int)

all games are valid now, we can proceed with the insertion

In [31]:
for i in range(len(c)):
    cur.execute(champions_table_insert, c[c.columns].values[i].tolist())
    conn.commit()

In [32]:
cur.execute("""SELECT * FROM champions Limit 3""")
cur.fetchall()

[(3190375139, 201, 13, 136, 2, 29, 497, 120, 117, 498, 121),
 (3191353856, 82, 99, 421, 38, 27, 55, 157, 497, 145, 59),
 (3191955256, 27, 117, 81, 56, 105, 145, 114, 201, 38, 72)]

<h1> #2: items Table </h1>

In [91]:
items = ['gameId','participantId','stats.item0', 'stats.item1', 'stats.item2', 'stats.item3', 'stats.item4', 'stats.item5', 'stats.item6']

In [92]:
df_items = df[items]

In [93]:
df_items['combine'] = df_items[['stats.item0', 'stats.item1', 'stats.item2', 'stats.item3', 'stats.item4', 'stats.item5', 'stats.item6']].values.tolist()
df_items = df_items[['gameId','participantId', 'combine']]
df_items['idx'] = df_items.groupby('gameId').cumcount()
df_items = df_items.pivot(index = 'gameId', columns='idx', values = 'combine').reset_index().rename_axis(columns=None, axis=1)

/var/folders/rs/xjf_6_vd4j16cdjg_j2td15h0000gn/T/ipykernel_27914/1770228038.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_items['combine'] = df_items[['stats.item0', 'stats.item1', 'stats.item2', 'stats.item3', 'stats.item4', 'stats.item5', 'stats.item6']].values.tolist()


In [94]:
df_items

,gameId,0,1,2,3,4,5,6,7,8,9
0,3.190375e+09,"[2055, 3401, 2423, 3024, 3801, 3117, 3364]","[3040, 2031, 3285, 3158, 2055, 0, 3340]","[2065, 2423, 2033, 3009, 3027, 1082, 3340]","[3047, 3800, 1413, 2055, 0, 0, 3364]","[1055, 3006, 2421, 3031, 1042, 1042, 3340]","[2421, 2031, 3096, 3117, 1028, 0, 3364]","[3057, 3111, 3044, 1042, 1036, 1036, 3340]","[0, 3070, 0, 3070, 3070, 3070, 0]","[1055, 2031, 3006, 3133, 1018, 1042, 3340]","[1412, 2031, 3117, 3134, 0, 0, 3364]"
1,3.191354e+09,"[3097, 3157, 0, 3020, 3116, 1082, 3364]","[3092, 2031, 3285, 3020, 2055, 0, 3364]","[3077, 3047, 1400, 1011, 1028, 3044, 3364]","[3003, 2420, 3020, 3027, 1054, 0, 3363]","[2033, 3800, 2065, 1026, 3047, 1082, 3340]","[2031, 1052, 3146, 3108, 3916, 3020, 3364]","[1055, 1083, 3006, 3046, 3031, 0, 3340]","[2055, 3050, 3092, 0, 0, 2422, 3364]","[0, 1055, 0, 3085, 3124, 3006, 3363]","[1412, 2031, 1031, 3111, 3071, 0, 3364]"
2,3.191955e+09,"[2033, 3194, 3075, 1082, 3047, 3151, 3340]","[3092, 3504, 3190, 3117, 0, 0, 3364]","[3153, 3140, 3070, 3042, 3025, 3158, 3363]","[1413, 1029, 3742, 3026, 3190, 3111, 3364]","[2033, 3157, 3152, 3100, 1082, 3158, 3364]","[1056, 2031, 3006, 3124, 3085, 3812, 3363]","[3074, 3078, 3111, 1055, 3065, 3133, 3340]","[3190, 3401, 3109, 3047, 1033, 2055, 3364]","[3040, 3157, 3135, 1054, 3027, 3111, 3364]","[3800, 3742, 3047, 3067, 1401, 1011, 3364]"
3,3.221805e+09,"[3098, 3050, 2055, 3801, 3117, 3114, 3364]","[2065, 3504, 2055, 3285, 1056, 2422, 3363]","[3117, 2031, 2055, 3078, 1412, 3052, 3340]","[3040, 2033, 3041, 3102, 3136, 1026, 3364]","[3144, 0, 1055, 3078, 3042, 3158, 3340]","[2031, 3508, 3006, 3094, 0, 1054, 3363]","[2031, 3057, 3044, 1042, 3047, 1412, 3364]","[3285, 1056, 3020, 3916, 1082, 1026, 3340]","[2031, 0, 0, 3022, 3071, 3111, 3340]","[2055, 3050, 3097, 2423, 0, 3117, 3364]"
4,3.227523e+09,"[2031, 3191, 3040, 3165, 3027, 3111, 3363]","[0, 3098, 3117, 3504, 3174, 3114, 3364]","[3153, 3031, 3094, 1055, 3047, 3071, 3363]","[0, 3026, 1412, 3078, 1028, 3047, 3364]","[3060, 2065, 3109, 3116, 3151, 3047, 3363]","[1056, 3157, 0, 3285, 3020, 3089, 3363]","[2055, 3071, 3047, 3095, 1412, 0, 3364]","[3111, 2055, 3401, 3067, 1031, 3050, 3364]","[3060, 3800, 2421, 3052, 3071, 3111, 3364]","[3033, 1054, 3047, 3072, 3095, 3086, 3363]"
...,...,...,...,...,...,...,...,...,...,...,...
87891,4.260971e+09,"[2421, 3373, 3083, 1031, 3111, 3379, 3363]","[3800, 3065, 3067, 2055, 1401, 3009, 3364]","[3389, 3071, 2421, 3091, 3111, 3156, 3340]","[3857, 2055, 3140, 3107, 3009, 3190, 3364]","[3389, 3156, 3006, 3123, 3085, 3072, 3363]","[1402, 3135, 3151, 2421, 3089, 3020, 3363]","[3179, 3864, 2055, 3814, 3006, 3033, 3364]","[2421, 3742, 3111, 0, 3076, 1031, 3364]","[3285, 3157, 3135, 3020, 3089, 0, 3363]","[3071, 3047, 3026, 2055, 3812, 3123, 3340]"
87892,4.260996e+09,"[2033, 3142, 3134, 3009, 3179, 0, 3363]","[3193, 3857, 2055, 1028, 0, 3117, 3364]","[1412, 2031, 2055, 3047, 3067, 3044, 3340]","[2033, 3152, 1056, 3020, 1082, 3108, 3364]","[3076, 2031, 0, 3068, 3047, 1056, 3364]","[3044, 3009, 3052, 3812, 1028, 1055, 3364]","[3860, 3050, 0, 1028, 0, 3117, 3364]","[1026, 3285, 3020, 1082, 3108, 3916, 3364]","[1055, 3031, 0, 1083, 3006, 3085, 3363]","[1413, 3109, 0, 3111, 2421, 3024, 3513]"
87893,4.261002e+09,"[2421, 3140, 3031, 3046, 3006, 3022, 3364]","[0, 3193, 3800, 1011, 3116, 3111, 3340]","[3153, 3140, 1053, 3078, 3042, 3158, 3363]","[3020, 1026, 3853, 2055, 3151, 0, 3363]","[1057, 0, 3194, 3742, 1401, 3111, 3364]","[3042, 3140, 3371, 3006, 3085, 3124, 3363]","[3388, 3071, 3053, 3033, 2421, 3047, 3340]","[1402, 3386, 3165, 3020, 3135, 1058, 3364]","[0, 3190, 3107, 3860, 3067, 3117, 3364]","[2420, 1031, 3047, 3379, 3373, 3025, 3340]"
87894,4.261012e+09,"[1055, 3071, 3812, 0, 3047, 0, 3364]","[2055, 2065, 3857, 2424, 1031, 3009, 3364]","[3115, 3006, 3042, 3124, 1055, 0, 3364]","[2031, 2055, 3165, 1082, 3020, 1402, 3364]","[3142, 1036, 3077, 3009, 3067, 3147, 3364]","[3860, 3800, 3067, 3117, 1029, 2055, 3364]","[3165, 3157, 0, 3020, 1054, 1026, 334

In [95]:
df_items.gameId = df_items.gameId.astype(int)

In [96]:
for i in range(len(df_items)):
    cur.execute(items_table_insert, df_items[df_items.columns].values[i].tolist())
    conn.commit()

In [97]:
cur.execute("""SELECT * FROM items limit 3""")
cur.fetchall()

[(4201095237,
  [0, 3748, 2055, 1055, 3047, 3071, 3340],
  [1082, 3030, 2033, 3111, 3165, 0, 3364],
  [1412, 3071, 0, 3047, 1028, 1028, 3364],
  [3031, 3046, 3139, 3026, 1055, 3006, 3340],
  [3853, 3801, 3174, 3504, 3117, 3114, 3364],
  [1029, 3748, 3071, 3047, 2055, 0, 3340],
  [1401, 3020, 2421, 3108, 0, 3165, 3364],
  [2033, 1026, 0, 3285, 1082, 3020, 3340],
  [3508, 3031, 0, 1055, 1055, 1001, 3363],
  [0, 3179, 3117, 3857, 2055, 3814, 3364]),
 (4201096497,
  [3031, 3508, 3086, 3140, 1055, 3006, 3363],
  [3153, 3139, 3006, 1038, 3071, 3133, 3363],
  [1400, 3026, 0, 3047, 1036, 3071, 3364],
  [3089, 3041, 2421, 3285, 3165, 3020, 3340],
  [3853, 3107, 3190, 2055, 3108, 3117, 3364],
  [2033, 3285, 3102, 3020, 3089, 0, 3363],
  [0, 3152, 3157, 1026, 3136, 3047, 3340],
  [1412, 2421, 2055, 3047, 3052, 3071, 3364],
  [3508, 3140, 3085, 1038, 1037, 3006, 3363],
  [3860, 3800, 0, 0, 0, 3117, 3364]),
 (4201102610,
  [3179, 3864, 2055, 3009, 3123, 1036, 3364],
  [3157, 3152, 2033, 1028, 1082,

<h1> #3: objectives_visions table </h1>

In [98]:
win_badra= winner_df[['baronKills', 'dragonKills', 'gameId']].rename(columns = {'baronKills' : 'winbaronKills','dragonKills': 'windragonKills' })
lose_badra= loser_df[['baronKills', 'dragonKills', 'gameId']].rename(columns = {'baronKills' : 'losebaronKills','dragonKills': 'losedragonKills' })

In [99]:
badra = pd.merge( win_badra, lose_badra, on=['gameId'], how = 'inner') 


In [100]:
badra = badra.groupby('gameId').mean()


In [101]:
col = ['stats.win','stats.wardsPlaced' , 'stats.wardsKilled', 'gameId','participantId' ]
df_ov = df[col]



In [102]:
df_false = df_ov[df_ov['stats.win'] == False]
df_true = df_ov[df_ov['stats.win'] == True]



In [103]:
lwp = df_false.groupby('gameId')['stats.wardsPlaced'].sum().to_frame().reset_index()
wwp = df_true.groupby('gameId')['stats.wardsPlaced'].sum().to_frame().reset_index()
lwk = df_false.groupby('gameId')['stats.wardsKilled'].sum().to_frame().reset_index()
wwk = df_true.groupby('gameId')['stats.wardsKilled'].sum().to_frame().reset_index()

In [105]:
from functools import reduce 



In [108]:
data_list = [lwp,wwp,lwk, wwk, a]



In [109]:
data_merge = reduce(lambda left, right:     # Merge DataFrames in list
                     pd.merge(left , right,
                              on = ["gameId"],
                              how = "inner"),
                     data_list)


In [110]:
data_merge = pd.merge( data_merge, badra, on=['gameId'], how = 'inner').drop(columns =['participants'] )

In [111]:
data_merge

,gameId,stats.wardsPlaced_x,stats.wardsPlaced_y,stats.wardsKilled_x,stats.wardsKilled_y,winbaronKills,windragonKills,losebaronKills,losedragonKills
0,3.190375e+09,26.0,33.0,3.0,5.0,0.0,0.0,0.0,1.0
1,3.191354e+09,59.0,70.0,17.0,23.0,0.0,0.0,1.0,2.0
2,3.191955e+09,66.0,79.0,21.0,21.0,0.0,0.0,1.0,3.0
3,3.221805e+09,46.0,51.0,10.0,9.0,0.0,1.0,1.0,1.0
4,3.227523e+09,81.0,90.0,30.0,33.0,1.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...
87891,4.260971e+09,98.0,95.0,46.0,38.0,0.0,4.0,1.0,0.0
87892,4.260996e+09,53.0,56.0,30.0,22.0,0.0,0.0,0.0,3.0
87893,4.261002e+09,109.0,155.0,69.0,51.0,0.0,2.0,2.0,4.0
87894,4.261012e+09,52.0,83.0,26.0,15.0,0.0,0.0,1.0,3.0


In [112]:
data_merge['win_baron_nashor'] = np.where(data_merge['winbaronKills'] > 0 , True, False)
data_merge['win_dragon_soul'] = np.where(data_merge['windragonKills'] >= 4 , True, False)
data_merge['lose_baron_nashor'] = np.where(data_merge['losebaronKills'] > 0 , True, False)
data_merge['lose_dragon_soul'] = np.where(data_merge['losedragonKills'] > 0 , True, False)

In [117]:
data_merge = data_merge.drop(columns =['winbaronKills', 'losebaronKills', 'windragonKills', 'losedragonKills' ])

In [118]:
data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87896 entries, 0 to 87895
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gameId               87896 non-null  float64
 1   stats.wardsPlaced_x  87896 non-null  float64
 2   stats.wardsPlaced_y  87896 non-null  float64
 3   stats.wardsKilled_x  87896 non-null  float64
 4   stats.wardsKilled_y  87896 non-null  float64
 5   win_baron_nashor     87896 non-null  bool   
 6   win_dragon_soul      87896 non-null  bool   
 7   lose_baron_nashor    87896 non-null  bool   
 8   lose_dragon_soul     87896 non-null  bool   
dtypes: bool(4), float64(5)
memory usage: 4.4 MB


In [119]:
for i in range(len(data_merge)):
    cur.execute(objectives_visions_table_insert, data_merge[data_merge.columns].values[i].tolist())
    conn.commit()


In [120]:
cur.execute("""SELECT * FROM objectives_visions limit 3""")
cur.fetchall()

[(3190375139, False, False, 33, 5, True, False, 26, 3),
 (3191353856, False, False, 70, 23, True, True, 59, 17),
 (3191955256, False, False, 79, 21, True, True, 66, 21)]

<h1> #4: champion_key table </h1>

In [131]:
champion_key_df = meta_champs[['data']]
champion_key_df = pd.json_normalize(champion_key_df['data'])

In [132]:
champion_key_df = champion_key_df[['key', 'id']].sort_values(by=['key'])


In [133]:
for i in range(len(champion_key_df)):
    cur.execute(champion_key_table_insert, champion_key_df[champion_key_df.columns].values[i].tolist())
    conn.commit()

In [134]:
cur.execute("""SELECT * FROM champion_key limit 3""")
cur.fetchall()

[(1, 'Annie'), (10, 'Kayle'), (101, 'Xerath')]

<h1> #4: item_key table </h1>

In [135]:
item_key_df = meta_items[['name']]
item_key_df['index'] = item_key_df.index


/var/folders/rs/xjf_6_vd4j16cdjg_j2td15h0000gn/T/ipykernel_27914/1959258733.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_key_df['index'] = item_key_df.index


In [137]:
for i in range(len(item_key_df)):
    cur.execute(item_key_table_insert, item_key_df[item_key_df.columns].values[i].tolist())
    conn.commit()

In [138]:
cur.execute("""SELECT * FROM item_key limit 3""")
cur.fetchall()

[(1001, 'Boots of Speed'), (1004, 'Faerie Charm'), (1006, 'Rejuvenation Bead')]

<h1> #4: game table </h1>

In [140]:
a2 = match_df[['gameId', 'participantIdentities', 'gameDuration', 'gameVersion']]
df = a2.explode('participantIdentities').reset_index(drop=True)
df = df.merge(pd.json_normalize(df['participantIdentities']), left_index=True, right_index=True)

In [141]:
df = df[['gameId', 'player.accountId']]


In [142]:
df = df.groupby('gameId')['player.accountId'].apply(list).reset_index()
a2 = a2[['gameId', 'gameDuration', 'gameVersion']]


In [143]:
game_df = pd.merge(df, a2, on = 'gameId', how = 'inner' )


In [149]:
game_df

,gameId,player.accountId,gameDuration,gameVersion
0,3.190375e+09,"[rKaQImjZTiOvZ4txjN-HVnLuhfjQlUNom9M13GwutBpi,...",1021.0,8.8.226.7254
1,3.191354e+09,[kaXDt1S1tNADQwlVpIsFi0Caz6N8lHFXJbTQJSR_SaJxI...,1507.0,8.9.227.7511
2,3.191955e+09,"[S9q18oVV-sIzC5_1j72PbGT6zByb6Qm1FBqOBu3bnbI, ...",1767.0,8.9.227.7511
3,3.221805e+09,"[7B11PtTVRsNoAWxRkgVFa03KkEW358aZk_-2xmr1VFa8,...",1358.0,8.10.229.7328
4,3.227523e+09,[x_vV5UDgtATjA3lpgRGO6dDWIxqt00MUKUr7SnBVouX8w...,2001.0,8.11.231.7304
...,...,...,...,...
87964,4.260971e+09,[6We8MBSW_YWaxbwNIQoim3goWBSwSAQVNYBYYYutfahf5...,2108.0,10.6.314.4405
87965,4.260996e+09,[kc7eS34jTXJ968Gm7vou_oo5v79EiRQI5FLGmQozCmH_X...,1250.0,10.6.314.4405
87966,4.261002e+09,[UpwIJ-o30d4lhkcPh7wE_6zdqYokH4uaPoZf1PBpsHWrD...,2190.0,10.6.314.4405
87967,4.261012e+09,[U2bQNAFp67-yvZQbdvHaIHdzIES8mpaYPml1patvMfp2x...,1538.0,10.6.314.4405


In [ ]:
for i in range(len(game_df)):
    cur.execute(games_table_insert, game_df[game_df.columns].values[i].tolist())
    conn.commit()

In [146]:
cur.execute("""SELECT * FROM games limit 3""")
cur.fetchall()

[(3190375139,
  1021.0,
  '8.8.226.7254',
  ['rKaQImjZTiOvZ4txjN-HVnLuhfjQlUNom9M13GwutBpi',
   'Th3f97roy4k8E__GshlFHDPfP76TLE5kptO4GecDx7by',
   '8abTvRQwXwzT0J4tCtEP_3-mHShfeAL3EcKAkfZA7EnC',
   '2j7W-hxT3CYCUSSzT3cbntQYry2bEx_ME_kybyV01KLy',
   'OeD4zr0xorGoAGlFXVHVo1hf474l7JqXhKXp67AEu7B0QVc',
   's5IirwYLAlm0bimWM6vV7eEWbJePZWWy0SywzLIqRLM',
   'qLbKO8QWzyy_ySveGsFFyTEtuRpSWU4R3MkH8SxE6iqc',
   'ej_Yq6ot3ORIHdw9YBpBPHXtcFitnD7bShFRrWvrK54F',
   'Pd7c7I2_IbRJKfKpxyH13YmkAYXQVtX9VCmn3yjU1hCX',
   'CTqmxBX1-b39i2eciDskPnWAbtu7I3cp7-HuUd12R8Cj']),
 (3191353856,
  1507.0,
  '8.9.227.7511',
  ['kaXDt1S1tNADQwlVpIsFi0Caz6N8lHFXJbTQJSR_SaJxIO8',
   'MHQRKpJwavPjK7DKYnayXCtW_5wpbqiYO05aytyYg2IanvI',
   'eFcFYgwwS6YbTSRrDnILkx_Nh4usIsQjOVyU7VMoJZlF',
   'eB-cQ7uhwmQNh5jpjh2sdDaEx9aUd9Kx-OV15zi81YLj',
   'agkXHV65QfgNjgAq38Mvm6bG3Zvm74YOWjnmqr0G1HCaZAY',
   'RMs8RAXjKsLSuSmCp7_F5TV3V5TUl41xqfYhc3M-XpEM',
   'GVZRA6dU4P9hMHLqjReX8COtBqMVuFeo7hErTIF3kCtU',
   'rKaQImjZTiOvZ4txjN-HVnLuhfjQlUNo

<h1> Close Connection to MOBAtatic Database <h2>

In [148]:
conn.close()
